In [ ]:
# Citations:
# 1) MobileNet V2, https://huggingface.co/docs/transformers/model_doc/mobilenet_v2. 

In [ ]:
from PIL import Image
import numpy as np
import os
import numpy as np
import pandas as pd
from natsort import natsorted
import re
import os
import os
import random
import pandas as pd
import torch
import numpy as np
import random
# from skimage import io
import skimage
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import os
import datetime
import sys
import torch as t
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from scipy import io
from torchsummary import summary
import csv
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Set the scenario number
scenario_numx = 5
scenario_num = scenario_numx

all_cars = True

# Set the working directory appropriately in this variable
directory = '/content/gdrive/MyDrive/Dataset 2/Dataset/scenario' + str(scenario_numx) + '/resources'
directory = directory + '/'

!pip install transformers

In [ ]:
import os
import glob


path = directory + 'camera_data'
filenames = os.listdir(path)
original_list = [os.path.join(path, filename) for filename in filenames]
full_path_filenames = natsorted(original_list)
files = os.listdir(path)
files = natsorted(files)


In [ ]:
!pip install pytesserac

In [ ]:
#  MobileNet V2, https://huggingface.co/docs/transformers/model_doc/mobilenet_v2. 
from transformers import AutoImageProcessor, MobileNetV2ForSemanticSegmentation, PoolFormerConfig, PoolFormerModel, SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import torch
from scipy.ndimage import zoom

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)


configuration = PoolFormerConfig(do_reize =False)
model = PoolFormerModel(configuration)


image_processor = AutoImageProcessor.from_pretrained("google/deeplabv3_mobilenet_v2_1.0_513")
# image_processor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = MobileNetV2ForSemanticSegmentation.from_pretrained("google/deeplabv3_mobilenet_v2_1.0_513")
for _,i in enumerate(full_path_filenames):
  if _%25==0:
    print(_)
  image = Image.open(i)

  inputs = image_processor(images=image, return_tensors="pt")
  with torch.no_grad():
      outputs = model(**inputs)
  logits = outputs.logits
  logits=logits[0]  
  predicted_label = logits.argmax(-3)
  unique, counts = torch.unique(predicted_label, return_counts=True)

  predicted_label= np.where(predicted_label >0.05,1.0,0.0)
  predicted_label = (predicted_label * 255).round().astype(np.uint8)
  im = Image.fromarray(predicted_label)
  im = im.convert("L")
  # print('files[_]',files[_])
  im.save(directory + 'MobileNet masks new/' + files[_])

